In [ ]:
from pymatgen.core import Structure
from mace.calculators import mace_mp

prim = Structure.from_file("Cs2SnI6_prim.cif")

# UPDATE TO USE RIGHT MACE MODEL
calc = mace_mp(model="medium", dispersion=False, default_dtype="float32", device='cuda')

In [ ]:
import mace


In [ ]:
# Importing list of text compounds from csv

import csv
import ast  # Safely parse strings representing Python an_ox_check structures

def convert_value(value):
    """Convert values to int, float, tuple, or keep as original string."""
    value = value.strip()  # Remove extra spaces

    try:
        # Attempt to parse tuple, list, int, or float
        parsed_value = ast.literal_eval(value)
        
        # Ensure that parsed values like lists/tuples are kept as they are
        if isinstance(parsed_value, (int, float, list, tuple)):
            return parsed_value
        else:
            return value  # If it's another type, return as string
    except (ValueError, SyntaxError):
        return value  # Return as string if it can't be parsed

# Read CSV and properly restore an_ox_check types



In [ ]:
from ase.filters import FrechetCellFilter
from ase.optimize import BFGS
from ase.io import write
import time
import os

num = [0,5481,4072,3858,2916] # Update this if the file name and length changes


phase = f"Phase 4 2.0" # Change here
tol_oct = []
with open('Phase 4 2.0 2916.csv', mode='r', newline='') as file:    # CHANGE IMPORT FILE HERE
    reader = csv.reader(file)
    header = next(reader)  # Read header
    tol_oct = [[convert_value(value) for value in row] for row in reader]  # Convert each value

# Output to confirm correct conversion
#print(header)
print((tol_oct[0]))  
print(len(tol_oct))

def MACE_program(compounds):

    start = time.process_time()
    start_2 = time.time()

    
    name = phase           # CHANGE NAME OF THE FOLDER
    # Ensure the results directory exists
    os.makedirs(f"{name}", exist_ok=True)

    
    for count, i in enumerate(compounds, start = 13412):       # Change starting number
        A = i[0]
        B = i[1]
        X = i[2]
        prim_volume = i[-1]
        
        structure = prim.copy()
        structure.replace_species(
            {
                "Cs": A,
                "Sn": B,
                "I": X,
            }
        )
    
        
        structure.scale_lattice(prim_volume)

        #print(A)
        
    
        atoms = structure.to_ase_atoms()
        #print(structure)

        atoms.calc = calc
        ecf = FrechetCellFilter(atoms)
        opt = BFGS(ecf)
        opt.run(fmax=0.01, steps=200)
        
        write(f"{name}/{str(atoms.symbols)}_{count}.xyz", atoms)

    
    end = time.process_time()
    end_2 = time.time()


    #print(f"Exact Anion List Length: {len(result_anions)}")
    #print(f"Remaining Oct List Length: {len(remaining_oct_list)}")
    print(f"CPU Time: {end - start} seconds")
    print(f"Elapsed Time: {end_2 - start_2} seconds")

    return

MACE_program(tol_oct)
    